### BO with 3 parameters ( ratio, time, temperature)
##### Data from 170 and 190 temperature are used as initial value

In [ ]:
import numpy as np
from skopt import Optimizer
import matplotlib.pyplot as plt
import copy
import math
from scipy.optimize import curve_fit

In [ ]:
# Set Range
x_max = 6.4
x_min = 0.0
x_gap = x_max-x_min
y_max = 60.0
y_min = 0.0
y_gap = y_max-y_min
z_min = 160.0
z_max = 200.0
z_gap = z_max-z_min
w_min = 0.0
w_max = 80.0
w_gap = w_max-w_min

In [ ]:
# Data collected by previous experiments(ratio, time, temperature, plqy)

x_190 = np.array([2.4,1.0,4.0,1.0,4.0,2.4,2.4,2.5,2.1,1.9,2.2,2.8,2.2,2.2,2.1,1.8])
y_190 = np.array([21.0,10.0,10.0,40.0,40.0,30.0,10.0,10.0,13.0,40.0,10.0,40.0,40.0,20.0,21.0,21.0])
z_190 = np.array([190.0,190.0,190.0,190.0,190.0,190.0,190.0,190.0,190.0,190.0,190.0,190.0,190.0,190.0,190.0,190.0])
w_190 = np.array([55.25,35.54,19.512,44.892,20.68,50.99,52.69,50.64,55.02,45.98,52.03,30.12,46.32,56.16,55.88,48.54])


x_170 = np.array([0.5,1.0,3.2,5.0,6.4,2.3,2.2,0.0,1.6,2.5,1.5,2.0,0.0,1.6,1.65,3.5,3.5,3.4,3.2,2.9,2.9,2.8,2.6,2.4,2.3])
y_170 = np.array([10.0,30.0,30.0,10.0,30.0,30.0,40.0,60.0,0.0,36.0,42.0,43.0,38.0,33.0,34.0,19.0,25.0,15.0,18.0,19.0,20.0,20.0,19.0,21.0,21.0])
z_170 = np.array([170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0])
w_170 = np.array([29.2,47.38,49.97,24.31,23.55,54.9,53.45,10.0,10.0,43.8,53.2,44.2,10.0,53.5,54.1,56.4,54.2,55.1,61.5,62.8,63.0,63.4,63.8,64.8,64.2])

x_temp = np.array([2.4,2.4,2.4,2.4,2.4])
y_temp = np.array([21.0,21.0,21.0,21.0,21.0])
z_temp = np.array([140.0,150.0,160.0,180.0,200.0])
w_temp = np.array([40.25,46.89,50.96,60.22,50.25])

In [ ]:
# Data collected by 3 parameter BO

x = np.hstack((x_170,x_190,[2.4,2.4,2.4,2.6,3.3,2.4,3.1,6.4,2.6,2.4,6.4]))
y = np.hstack((y_170,y_190,[21.0,21.0,21.0,19.0,20.0,20.0,21.0,60.0,18.0,18.0,60.0]))
z = np.hstack((z_170,z_190,[180.0,160.0,200.0,174.0,161.0,172.0,176.0,200.0,174.0,173.0,160.0]))
w = np.hstack((w_170,w_190,[60.22,50.96,50.25,63.91,49.18,65.41,61.14,10.0,64.10,65.58,10.0]))

# Set of initial values
rx = (x-x_min)/x_gap
ry = (y-y_min)/y_gap
rz = (z-z_min)/z_gap
rw = (w-w_min)/w_gap
print(rz)

In [ ]:
# Draw Guassian Process mean prediction graph

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

x_test = np.vstack((np.linspace(0.0,1.0,21),np.full((1,21),21.0/y_max),np.full((1,21),(170.0-z_min)/z_gap))).T
x_train = np.vstack((rx,ry,rz)).T
kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel)
gaussian_process.fit(x_train, rw)
gaussian_process.kernel_

In [ ]:
mean_prediction, std_prediction = gaussian_process.predict(x_test, return_std=True)
print(x_test[:,0])
plt.plot(x_test[:,0], mean_prediction, label="Mean prediction")
plt.plot((x_170-x_min)/x_gap,(w_170-w_min)/w_gap,'o',label="Observation")
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("")

In [ ]:
# Proceed BO with 3 parameters
# Get 10 recommedations
bounds = [(0.0, 1.0),(0.0, 1.0),(0.0,1.0)]
epoch = 10
result = np.array([[0.0,0.0,0.0] for i in range(epoch)])
acq_func_kwargs = {"xi": 0.000001}
for k in range(epoch):
    if(k%1==0):
        print("epoch : "+str(k))
    opt_r = Optimizer(bounds, base_estimator="gp",n_initial_points = 0)
    for i in range(0,len(rx)):
        opt_r.tell([rx[i],ry[i],rz[i]],-rw[i])
    next_rx = opt_r.ask(n_points = 1)
    for i in next_rx:
        result[k][0]=i[0]*x_gap+x_min
        result[k][1]=i[1]*y_gap+y_min
        result[k][2]=i[2]*z_gap+z_min

In [ ]:
print(result)
print(result[0][0])